In [ ]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import re
data = []
page_number = 1

#for each page
while True:
  url = f"https://meqasa.com/houses-for-rent-in-ghana?w={page_number}"
  source = requests.get(url)
  soup = BeautifulSoup(source.content, "lxml")
  #Set base_url
  base_url = "https://meqasa.com"
  listing = soup.find("div", class_ = "filtRpg").find_all("div", class_ = "mqs-prop-dt-wrapper")

  #for each house
  for house_details in listing:
    property_name = house_details.find("a").text
    house_url = base_url + house_details.h2.a['href']
    price = house_details.find("p", class_ = "h3").span.next.next
    #use regex to take off the currency from price
    price_only = re.sub(r"[^0-9]", '', str(price))
    try:
      rent_period =  house_details.find("p", class_ = "h3").span.next.next.next.text.replace("/ ", '')
    except:
      rent_period = "N/A"
    bed = house_details.ul.find("li", class_ = "bed").text
    try:
      garage = house_details.ul.find("li", class_ = "garage").text # get the garage
    except:
      garage = "0"
    try:
      area = house_details.ul.find("li", class_ = "area").text
    except:
      area = "N/A"
    try:
      shower = house_details.ul.find("li", class_ = "shower").text
    except:
      shower = "N/A"
    # Use regular expression to extract currency from price
    currency = re.sub(r"[0-9,]", '', str(price)).strip()
    # go to individual links to get address, description and time posted
    house_page = requests.get(house_url)
    house_soup = BeautifulSoup(house_page.content, 'lxml')
    try:
      description = house_soup.find("div", class_ = "description").p.text
    except: 
      description = "N/A"
    try:
      time_posted = " ".join(house_soup.find("aside").p.text.split(" ")[2:5])
    except:
      time_posted = "N/A"
    for td in house_soup.find_all('tr'):
      if td.find_all("td")[0].text == "Address":
        address = td.find_all("td")[1].text
    
    data.append({
    "property": property_name,
    "beds": bed,
    "showers": shower,
    "garages": garage,
    "area" : area,
    "description": description,
    "price": price_only,
    "currency": currency,
    "rent_period": rent_period,
    "url": house_url,
    "address": address,
    "time_posted": time_posted,
    "page_number": page_number
    })
    
  page_number += 1
  if page_number > 1:
    break
df = pd.DataFrame(data)
df.to_csv("house_rent_all.csv")

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("house_rent_all.csv")

In [ ]:
#!jupyter nbconvert --to script "/content/drive/MyDrive/Colab Notebooks/house_rent.ipynb"